# Árboles de decisión y Random Forest
### Valeria Pérez Cong Sánchez
### 145009
### November 29, 2017

Brief description: Compare the performance of a random forest with the performance of a decision tree

## El set de datos

El set de datos que vamos a ocupar es un set de datos súper típico. Es el set de datos de Iris que se puede encontrar en UCI Machine Learning. El objetivo es encontrar a qué tipo de clasificación pertenece una flor (hay 3 clasificaciones en total) según 4 atributos. La ventaja del set es que está balanceado y hay 50 observaciones por cada clase (es decir, un total de 150 observaciones en total). 

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import tree
from sklearn import datasets

iris = datasets.load_iris()
type(iris)

sklearn.datasets.base.Bunch

In [2]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

Los 4 atributos que se usarán para clasificar a las flores son de tipo númerico. 

In [3]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Se puede observar que la variable target tiene 3 valores diferentes,uno para cada clasificación: 0,1,2. 

## Dividimos el set en entrenamiento y pruebas

Se usará 70% para entrenar el modelo y el 30% para pruebas

In [14]:
from sklearn.cross_validation import train_test_split

from sklearn import tree

X_train, X_test, Y_train, Y_test = train_test_split(iris.data, iris.target, test_size=0.30, random_state=42)

Uno de los parámetros que se debe establecer es el min_samples_leaf, que es el número de observaciones para que un árbol deje de dividir el set. Si el número es muy chico, es posible que se este sobreentrenando al modelo, si el número es muy grande es posible que el modelo esté demasiado generalizado. 

## Árbol con 4 min_samples_leaf

In [15]:
#Entrenamos el modelo
tree_class = tree.DecisionTreeClassifier(min_samples_leaf=4)
tree_class = tree_class.fit(X_train, Y_train)

In [16]:
# importancia de cada variable
tree_class.feature_importances_

array([ 0.00691913,  0.        ,  0.58248457,  0.4105963 ])

Se puede observar que el atributo 2 (sepal width) tiene una importancia de 0 según el criterio Gini (que viene por default y que se refiere al grado de pureza/ homogeneidad de los nodos) 

### Visualización del árbol

In [17]:
from sklearn.externals.six import StringIO  
import pydotplus

dot_data = StringIO() 
tree.export_graphviz(tree_class, out_file=dot_data) 
graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 
graph.write_png("iris.png") 

True

![iris_decision_tree](/Users/valeriaperezcong/Desktop/ITAM/9/aprendizaje/udemy/DataScience-Python3/decision_trees/iris.png)

![tree](iris.png "Iris Tree")

### Predicciones con el set de prueba

In [18]:
prediction = tree_class.predict(X_test)

### Medidas de desempeño

In [19]:
## valores reales
Y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0,
       2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0])

In [20]:
## valores predichos
prediction

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0,
       2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0])

A simple vista, solo de ver los dos arrays del valor real y el valor predicho, el modelo acertó en todas las observaciones del set de pruebas 

### Matriz de confusión

In [21]:
## matriz de confusión
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(Y_test, prediction)
confusion

array([[19,  0,  0],
       [ 0, 13,  0],
       [ 0,  0, 13]])

### Accuracy

In [22]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_test, prediction)

1.0

### Precision, recall y F1-Measure

In [23]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(Y_test, prediction, target_names=target_names))

             precision    recall  f1-score   support

    class 0       1.00      1.00      1.00        19
    class 1       1.00      1.00      1.00        13
    class 2       1.00      1.00      1.00        13

avg / total       1.00      1.00      1.00        45



El modelo clasificó correctamente todas las observaciones del test set :)

## Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
#dejamos todos los valores por default, pero decidimos tener 20 árboles. 
rf = RandomForestClassifier(n_estimators=50)

rf=rf.fit(X_train, Y_train)

print(rf.feature_importances_)

[ 0.09564722  0.03492283  0.39839304  0.47103691]


Se puede observar que la importancia que se le da a cada variable es muy distinta a la que el árbol daba:

array([ 0.00691913,  0.    ,  0.58248457,  0.4105963 ])

### Predicciones con el set de pruebas

In [28]:
prediction_rf = rf.predict(X_test)

### Matriz de confusión

In [29]:
confusion_rf = confusion_matrix(Y_test, prediction_rf)
confusion_rf

array([[19,  0,  0],
       [ 0, 13,  0],
       [ 0,  0, 13]])

Se puede observar que es igual a la matriz de confusión del árbol de decisión.

### Medidas de desempeño

In [30]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_test, prediction_rf)

1.0

In [31]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(Y_test, prediction_rf, target_names=target_names))

             precision    recall  f1-score   support

    class 0       1.00      1.00      1.00        19
    class 1       1.00      1.00      1.00        13
    class 2       1.00      1.00      1.00        13

avg / total       1.00      1.00      1.00        45



### Tanto el árbol de decisión como el modelo Random Forest tuvieron un desempeño perfecto. 